In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import collections
from itertools import combinations

In [4]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1026,tt1645170,65000000,179379533,The Dictator,Sacha Baron Cohen|Sayed Badreya|Aasif Mandvi|M...,Larry Charles,kurd,The heroic story of a dictator who risks his l...,83,Comedy,Paramount Pictures,5/15/2012,5.9,2012
1364,tt0332047,30000000,50451307,Fever Pitch,Drew Barrymore|Jimmy Fallon|Jason Spevack|Jack...,Bobby Farrelly|Peter Farrelly,A Comedy About The Game of Love.,When relaxed and charming Ben Wrightman meets ...,103,Comedy|Drama|Romance,Fox 2000 Pictures|Wildgaze Films|Alan Greenspa...,4/6/2005,5.7,2005
1199,tt0359950,90000000,188133322,The Secret Life of Walter Mitty,Ben Stiller|Kristen Wiig|Patton Oswalt|Shirley...,Ben Stiller,"Stop Dreaming, Start Living",A timid magazine photo manager who lives life ...,114,Adventure|Comedy|Drama|Fantasy,New Line Cinema|Ingenious Media|Twentieth Cent...,12/18/2013,7.0,2013
199,tt0787474,60000000,108255770,The Boxtrolls,Ben Kingsley|Isaac Hempstead Wright|Elle Fanni...,Graham Annable|Anthony Stacchi,When troubles strikes friends stack together,An orphaned boy raised by underground creature...,97,Animation|Comedy|Family|Fantasy,Laika Entertainment,9/10/2014,6.6,2014
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


In [5]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [7]:
#создаем словарь для ответов
answers = {} 

#добавляем столбец с прибылью 

data['profit'] = data['revenue'] - data['budget']

#добавляем новую колонку с длиной названий фильмов

data['title_len'] = data['original_title'].map(lambda s: len(s))

#добавляем новую колонку с обзором

data['overview_len'] = data['overview'].map(lambda s: len(s.split()))

#добавляем новый столбец с режиссерами

data['dir_list'] = data.director.str.split('|')

#добавляем новый столбец с жанрами

data['genres_list'] = data.genres.str.split('|')

#добавляем новый столбец со списком актеров

data['cast_list'] = data.cast.str.split('|')

#добавляем новый столбец с названиями компаний

data['companies_list'] = data.production_companies.str.split('|')

#разделяем дату выпуска

data['month'] = [int(x.split('/')[0]) for x in data['release_date']]

## 1. У какого фильма из списка самый большой бюджет?

In [15]:
max_budget = data[data.budget == data.budget.max()].original_title
print(max_budget)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


## 2. Какой из фильмов самый длительный (в минутах)?

In [16]:
longest_movie = data[['imdb_id','runtime','original_title']].sort_values(['runtime'],ascending=False).head(1)
print(longest_movie)

        imdb_id  runtime     original_title
1157  tt0279111      214  Gods and Generals



## 3. Какой из фильмов самый короткий (в минутах)?

In [17]:
shortest_movie = data[['imdb_id','runtime','original_title']].sort_values(['runtime'],ascending=True).head(1) 
print(shortest_movie)

       imdb_id  runtime   original_title
768  tt1449283       63  Winnie the Pooh


## 4. Какова средняя длительность фильмов?

In [18]:
mean_runtime = round(data.runtime.mean())   
print(mean_runtime)

110


## 5. Каково медианное значение длительности фильмов?

In [19]:
median_runtime = data.runtime.median() 
print(int(median_runtime))

107


## 6. Какой самый прибыльный фильм?

In [20]:
#используем столбец с прибылью
profitable_movie = data[['imdb_id','original_title','profit', 'budget', 'revenue']].sort_values(['profit'],ascending=False).head(1) 
print(profitable_movie)

       imdb_id original_title      profit     budget     revenue
239  tt0499549         Avatar  2544505847  237000000  2781505847


## 7. Какой фильм самый убыточный?

In [21]:
unprofitable_movie = data[['imdb_id','original_title','profit', 'budget', 'revenue']].sort_values(['profit'],ascending=True).head(1) 
print(unprofitable_movie)

        imdb_id   original_title     profit     budget   revenue
1245  tt1210819  The Lone Ranger -165710090  255000000  89289910


## 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
profits_number = data[data.profit >0]
print(len(profits_number))

1478


## 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
highest_grossing = data[data.release_year == 2008].sort_values(['revenue'],ascending=False).head(1) 
highest_grossing

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,vote_average,release_year,profit,title_len,overview_len,dir_list,genres_list,cast_list,companies_list,month
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,8.1,2008,816921825,15,67,[Christopher Nolan],"[Drama, Action, Crime, Thriller]","[Christian Bale, Michael Caine, Heath Ledger, ...","[DC Comics, Legendary Pictures, Warner Bros., ...",7


## 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [24]:
years12_14 = data.query('2012<=release_year<=2014')
unprofitable12_14 = years12_14[years12_14.profit == years12_14.profit.min()].original_title
print(unprofitable12_14)

1245    The Lone Ranger
Name: original_title, dtype: object


## 11. Какого жанра фильмов больше всего?

In [25]:
print(pd.Series(data['genres'].str.cat(sep='|').split('|')).value_counts().head(1))

Drama    782
dtype: int64


## 12. Фильмы какого жанра чаще всего становятся прибыльными?

In [26]:
print(pd.Series(profits_number['genres'].str.cat(sep='|').split('|')).value_counts().head(1))

Drama    560
dtype: int64


## 13. У какого режиссера самые большие суммарные кассовые сборы?

In [27]:
grossing_director = data.groupby(['director'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1)
print(grossing_director)

                  revenue
director                 
Peter Jackson  6490593685


## 14. Какой режисер снял больше всего фильмов в стиле Action?

In [28]:
action_movies = data[data.genres.str.contains ("Action", na=False)]
dir_exploded = action_movies.explode('dir_list')
action_director = dir_exploded.dir_list.value_counts().head(1)
print(action_director)

Robert Rodriguez    9
Name: dir_list, dtype: int64


## 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [29]:
year_2012 = data[data.release_year == 2012]
exploded_actor = year_2012.explode('cast_list')
grossing_actor = exploded_actor.groupby(['cast_list'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1)
print(grossing_actor)

                    revenue
cast_list                  
Chris Hemsworth  2027450773


## 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [30]:
budget_movie = data[data.budget > data.budget.mean()]
exploded_actor2 = budget_movie.explode('cast_list')
grossing_actor2 = exploded_actor2.cast_list.value_counts().head(1)
print(grossing_actor2)

Matt Damon    18
Name: cast_list, dtype: int64


## 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [31]:
Cage = data[data.cast.str.contains ("Nicolas Cage", na=False)]
print(pd.Series(Cage['genres'].str.cat(sep='|').split('|')).value_counts().head(1))

Action    17
dtype: int64


## 18. Самый убыточный фильм от Paramount Pictures?

In [32]:
Paramount = data[data.production_companies.str.contains ("Paramount Pictures", na=False)]
unprofitable_Paramount = Paramount[['imdb_id','original_title','profit']].sort_values(['profit'],ascending=True).head(1) 
print(unprofitable_Paramount)

       imdb_id        original_title    profit
925  tt0267626  K-19: The Widowmaker -64831034


## 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [33]:
best_year = data.groupby(['release_year'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1)
print(best_year)

                  revenue
release_year             
2015          25449202382


## 20. Какой самый прибыльный год для студии Warner Bros?

In [34]:
Warner = data[data.production_companies.str.contains ("Warner Bros", na=False)]
best_Warner = Warner.groupby(['release_year'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1) 
print(best_Warner)

                 revenue
release_year            
2014          3243064519


## 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [35]:
pivot_months = data.pivot_table(values='imdb_id',
                           index='release_year',
                           columns='month',
                           fill_value=0,
                           aggfunc='count')
display(pivot_months.sum().sort_values(ascending=False).head(1))

month
9    227
dtype: int64

## 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [36]:
display(pivot_months.sum()[5:8].sum())

450

## 23. Для какого режиссера зима – самое продуктивное время года?

In [37]:
winter = data[(data.month.isin([1,2,12]))] 
winter_dir = winter.director.str.split('|').sum() 
display(Counter(winter_dir).most_common(1))

[('Peter Jackson', 7)]

## 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [38]:
exploded_companies = data.explode('companies_list')
title_names = exploded_companies.groupby(['companies_list'])['title_len'].mean().sort_values(ascending=False)
print(title_names.head(1))

companies_list
Four By Two Productions    83.0
Name: title_len, dtype: float64


## 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [39]:
longest_overview = exploded_companies.groupby(['companies_list'])['overview_len'].mean().sort_values(ascending=False)
print(longest_overview.head(1))

companies_list
Midnight Picture Show    175.0
Name: overview_len, dtype: float64


## 26. Какие фильмы входят в 1 процент лучших по рейтингу?

In [40]:
top = data[data['vote_average'] > data['vote_average'].quantile(0.99)][['original_title','vote_average']]
print(top)

                                     original_title  vote_average
9                                        Inside Out           8.0
34                                             Room           8.0
118                                    Interstellar           8.0
119                         Guardians of the Galaxy           7.9
125                              The Imitation Game           8.0
128                                       Gone Girl           7.9
138                        The Grand Budapest Hotel           7.9
370                                       Inception           7.9
599                                 The Dark Knight           8.1
872                                     The Pianist           7.9
1081  The Lord of the Rings: The Return of the King           7.9
1183                        The Wolf of Wall Street           7.9
1191                               12 Years a Slave           7.9
1800                                        Memento           7.9


## 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [41]:
data['pairs'] = data['cast_list'].apply(lambda s: list(combinations(s, 2)))
data = data.explode('pairs')
collections.Counter(data['pairs']).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]